In [1]:
DATA_DIR = 'data/'
RESULT_DIR = 'results/original_vs_updated/'
import os
# Create the directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [2]:
import pandas as pd 
import wandb
from dotenv import load_dotenv
load_dotenv(override=True)
api = wandb.Api()


# Project is specified by <entity/project-name>
entity = os.getenv("WANDB_ENTITY", "dl2-2024")
project = os.getenv("WANDB_PROJECT", "dl-2024")
runs = api.runs(f"{entity}/{project}")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)
    summary_list[-1]['tags'] = run.tags

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv(DATA_DIR + "project.csv")

In [3]:
import pandas as pd
from tabulate import tabulate

# Read the CSV file
runs_df = pd.read_csv(DATA_DIR + "project.csv")

# Extract and flatten the summary and config columns
summary_df = pd.json_normalize(runs_df['summary'].apply(eval))
config_df = pd.json_normalize(runs_df['config'].apply(eval))

# Combine the flattened summary and config data
results_df = pd.concat([runs_df[['name']], config_df, summary_df], axis=1)

# Filter results based on `softmax`, `use_underscore`, and `full_val` settings
original_code_full_val = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True) & (results_df['full_val'] == True) & (results_df['full_val_pf'] == False)]
updated_code_full_val = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False) & (results_df['full_val'] == True) & (results_df['full_val_pf'] == False)]

# Function to get the specified metric for each dataset, model, and transformation combination
def get_metric(df, dataset_name, model_name, transformation, metric):
    result = df[(df['dataset_name'] == dataset_name) & 
                (df['model_name'] == model_name) & 
                (df['data_transformations'] == transformation)][metric]
    return round(result.iloc[0],2) if not result.empty else None

datasets = ['CIFAR100']  # Add your datasets here
metrics = ["val_top1_acc", "final_top1_acc"]  
transformations = ['rot90', 'flip']
architectures = ['RN50']  # Add more architectures if needed

# Create a summary DataFrame for each dataset
for i, dataset in enumerate(datasets):
    summary_data_full_val = {
        "Method": [],
        "Architecture-Transformation": [],
        "Prefinetune Top1 Acc": [],
        "Finetune Top1 Acc": []
    }

    # Populate the summary data
    for architecture in architectures:
        for transformation in transformations:
            combination = f"CLIP w {architecture} - {transformation} - *λ-equitune*"
            
            # Original code
            summary_data_full_val["Method"].append("Original Code")
            summary_data_full_val["Architecture-Transformation"].append(combination)
            summary_data_full_val["Prefinetune Top1 Acc"].append(
                get_metric(original_code_full_val, dataset, architecture, transformation, metrics[0]))
            summary_data_full_val["Finetune Top1 Acc"].append(
                get_metric(original_code_full_val, dataset, architecture, transformation, metrics[1]))
            
            # Updated code
            summary_data_full_val["Method"].append("Updated Code")
            summary_data_full_val["Architecture-Transformation"].append(combination)
            summary_data_full_val["Prefinetune Top1 Acc"].append(
                get_metric(updated_code_full_val, dataset, architecture, transformation, metrics[0]))
            summary_data_full_val["Finetune Top1 Acc"].append(
                get_metric(updated_code_full_val, dataset, architecture, transformation, metrics[1]))

    summary_df_full_val = pd.DataFrame(summary_data_full_val)

    # Convert the dataframe to a markdown table
    markdown_table_full_val = tabulate(summary_df_full_val, headers='keys', tablefmt='pipe')

    # Add a caption to the markdown table
    markdown_table_full_val = f"## Original vs Updated Full Val Summary for {dataset}\n\n{markdown_table_full_val}"
    
    # Add a note to the table
    markdown_table_full_val += f"\n\n*Table {i+1}: Image classification results using the author's original and our modified code base*"

    # Save the markdown table to a file
    with open(f"{RESULT_DIR}{dataset}_original_vs_updated_full_val_summary.md", "w") as file:
        file.write(markdown_table_full_val)

    # Display the markdown table
    print(markdown_table_full_val)


## Original vs Updated Full Val Summary for CIFAR100

|    | Method        | Architecture-Transformation        |   Prefinetune Top1 Acc |   Finetune Top1 Acc |
|---:|:--------------|:-----------------------------------|-----------------------:|--------------------:|
|  0 | Original Code | CLIP w RN50 - rot90 - *λ-equitune* |                  31.42 |               51.17 |
|  1 | Updated Code  | CLIP w RN50 - rot90 - *λ-equitune* |                  35.12 |               56.15 |
|  2 | Original Code | CLIP w RN50 - flip - *λ-equitune*  |                  37.07 |               54.04 |
|  3 | Updated Code  | CLIP w RN50 - flip - *λ-equitune*  |                  37.69 |               55.64 |

*Table 1: Image classification results using the author's original and our modified code base*
